<a href="https://colab.research.google.com/github/hailusong/nlp-qa/blob/master/nlp_textsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Text Summarization**
Refs:
- https://huggingface.co/models
- https://huggingface.co/models?search=bertabs
- [BertForMaskedLM](https://github.com/huggingface/transformers/blob/13aa174112f0c2ee794c44188ecf13b241694db0/src/transformers/modeling_bert.py#L876) used as architecture in BertAbs model, which can do
  * **masked language modeling** loss, for language modeling, or
  * left-to-right language modeling loss (**next word prediction**)

Hyper-params:
TBA

Issues:
1. Max sentence length: 64 (truncated)

## Environment Setup
To get the source code of Huggingface transformer release, do this (assuming the release version is 2.11.0)
```
git fetch -a --tags
git checkout tags/v2.11.0
```

Make sure we have access to file command in Linux

In [1]:
!uname -a
!pip install wget
!apt-get install file

Linux 49ba081d07d6 4.19.104+ #1 SMP Wed Feb 19 05:26:34 PST 2020 x86_64 x86_64 x86_64 GNU/Linux
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=85bdb9dccbca33b9797cdc19b61272b80f3a11cfa2e42bbd00a89348f11719f9
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68

In [2]:
try:
    import transformers
except ImportError as e:
    # install huggingface
    !pip install transformers

     |████████████████████████████████| 675kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 15.8MB/s 
     |████████████████████████████████| 890kB 17.9MB/s 
     |████████████████████████████████| 3.8MB 23.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7f07fbea916716cf847951323fc37f280e07c56b7b249ad49a5e6b44087b7e87
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

1.5.0+cu101
2.11.0


## Configuratiuon

In [0]:
EXAMPLE_MODE=True
INLINE_MODE=False

In [5]:
if EXAMPLE_MODE:
  # ! [ ! -d "./transformers" ] && git clone --depth 1 https://github.com/huggingface/transformers
  # ! [ -d "./transformers" ] && \
  #   cd transformers && \
  #   git fetch -a --tags && \
  #   git checkout tags/v2.11.0 && \
  #   pip install . && \
  !rm -rf ./nlp-qa/
  ![ ! -d "./nlp-qa" ] && git clone --depth 1 https://github.com/hailusong/nlp-qa.git
  !pip install nltk py-rouge

Cloning into 'nlp-qa'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 45 (delta 2), reused 29 (delta 2), pack-reused 0
Unpacking objects: 100% (45/45), done.
     |████████████████████████████████| 61kB 1.8MB/s 


### GPU Setup

In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## Download

### Pre-trained BERTABS fine-tuned on CNN/DM corpus 


In [0]:
if INLINE_MODE:
  # from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  # from transformers import AlbertConfig, AlbertForSequenceClassification
  # import torch
  from modeling_bertabs import BertAbs, build_predictor
  from transformers import BertTokenizer

  tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
  model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
  model.to(args.device)
  model.eval()

### Text to be summarized

In [8]:
if EXAMPLE_MODE:
  %env DATA_PATH=/content/nlp-qa/stories
  %env SUMMARIES_PATH=/content/textsum-out

  !mkdir -p $SUMMARIES_PATH
  !ls -al $SUMMARIES_PATH


env: DATA_PATH=/content/nlp-qa/stories
env: SUMMARIES_PATH=/content/textsum-out
total 8
drwxr-xr-x 2 root root 4096 Jun 10 13:25 .
drwxr-xr-x 1 root root 4096 Jun 10 13:25 ..


## Inference
Source: https://pytorch.org/hub/huggingface_pytorch-transformers/


### BertAbs

In [9]:
if EXAMPLE_MODE:
  # !pwd
  # !ls -al transformers/examples/summarization/bertabs
  # !cd transformers/examples/summarization/bertabs && python run_summarization.py \
  #   --documents_dir $DATA_PATH \
  #   --summaries_output_dir $SUMMARIES_PATH \
  #   --no_cuda false \
  #   --batch_size 4 \
  #   --min_length 50 \
  #   --max_length 200 \
  #   --beam_size 5 \
  #   --alpha 0.95 \
  #   --block_trigram true
  # !ls -al nlp-qa/textsum/bertabs
  !cd nlp-qa/textsum/bertabs && python run_summarization.py \
    --documents_dir $DATA_PATH \
    --summaries_output_dir $SUMMARIES_PATH \
    --no_cuda false \
    --batch_size 4 \
    --min_length 20 \
    --max_length 50 \
    --beam_size 5 \
    --alpha 0.95 \
    --block_trigram true

2020-06-10 13:25:29.345043: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
INFO:filelock:Lock 140646512241520 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpo33_kwov
Downloading: 100% 232k/232k [00:00<00:00, 4.18MB/s]
INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt in cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/tran

### BART

In [0]:
if EXAMPLE_MODE:
  !cd nlp-qa/textsum/bertabs && python evaluate_cnn.py $DATA_PATH $SUMMARIES_PATH/bart_cnn_test_summaries.txt

### Result

In [10]:
!ls -al $SUMMARIES_PATH
!grep "" $SUMMARIES_PATH/*

total 40
drwxr-xr-x 2 root root 4096 Jun 10 13:34 .
drwxr-xr-x 1 root root 4096 Jun 10 13:25 ..
-rw-r--r-- 1 root root  206 Jun 10 13:34 e117408ad19cc69e15b1e21b9ae54f10c07223ce_summary.story
-rw-r--r-- 1 root root  252 Jun 10 13:34 e3dd06d326c8d53722bdc5c8428e29c68a60d0d5_summary.story
-rw-r--r-- 1 root root  228 Jun 10 13:34 e9093ca0a82f2aba28cd2762942c04177cfbb000_summary.story
-rw-r--r-- 1 root root  198 Jun 10 13:34 ea06fd0b25cb9793397a51de73fd83f91b4323fa_summary.story
-rw-r--r-- 1 root root  243 Jun 10 13:34 eb68bc51ed4fc727a1af058192a3fef0916c91e7_summary.story
-rw-r--r-- 1 root root  267 Jun 10 13:34 ee0ba7928d8987f2cb21c9a2012a76730f77de45_summary.story
-rw-r--r-- 1 root root  218 Jun 10 13:34 ee8871b15c50d0db17b0179a6d2beab35065f1e9_summary.story
-rw-r--r-- 1 root root  255 Jun 10 13:34 mine1_summary.story
/content/textsum-out/e117408ad19cc69e15b1e21b9ae54f10c07223ce_summary.story:the partnership is an organization with partners throughout the uk. it employs approximately 67